In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from pathlib import Path
import re
from typing import Any
from pydantic import Field

from grasp_agents.grasp_logging import setup_logging
from grasp_agents.agent_message_pool import (
    AgentMessagePool,
)
from grasp_agents.llm_agent import LLMAgent
from grasp_agents.openai.openai_llm import (
    OpenAILLM,
    OpenAILLMSettings,
)
from grasp_agents.typing.io import (
    AgentPayload,
    LLMPromptArgs,
)
from pydantic import BaseModel
from grasp_agents.typing.tool import BaseTool
from grasp_agents.run_context import RunContextWrapper, RunArgs
from grasp_agents.agent_message import AgentMessage
from grasp_agents.typing.content import ImageData
from grasp_agents.utils import get_timestamp
from grasp_agents.workflow.sequential_agent import SequentialWorkflowAgent
from grasp_agents.llm_agent_state import LLMAgentState

/Users/xf/Work/grasp/grasp-agents/src/grasp_agents/utils.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
PACKAGE_DIR = Path.cwd()
DATA_DIR = Path.cwd() / "data/multiagent"

In [4]:
LOGGING_DIR = DATA_DIR / "logs"
LOGGING_CFG_PATH = PACKAGE_DIR / "configs/logging/default.yaml"
setup_logging(
    LOGGING_DIR / f"agents_demo_{get_timestamp()}.log",
    LOGGING_CFG_PATH,
)

# Chat

In [5]:
IMG_1_URL = "https://www.simplilearn.com/ice9/free_resources_article_thumb/Expressions_In_C_2.PNG"
IMG_2_PATH = Path.cwd() / "src/grasp_agents/examples/data/expr.jpeg"

In [6]:
class Response(AgentPayload):
    response: str


chatbot = LLMAgent[Any, Response, None](
    agent_id="chatbot",
    llm=OpenAILLM(
        model_name="gpt-4.1",
        llm_settings=OpenAILLMSettings(),
    ),
    sys_prompt=None,
    out_schema=Response,
)


@chatbot.parse_output_handler
def output_handler(conversation, ctx, **kwargs) -> Response:
    return Response(response=conversation[-1].content)


# This initialises printer and usege tracker
ctx = RunContextWrapper(print_messages=True)

In [7]:
out = await chatbot.run("Where are you headed, stranger?", ctx=ctx)
print(out.payloads[0].response)


<chatbot>[USER]
Where are you headed, stranger?

<chatbot>[ASSISTANT]
Nowhere in particular—just wherever the next question takes me! Where do you want to go?

------------------------------------
I/O/(R)/(C) tokens: 14/21/0/0
Nowhere in particular—just wherever the next question takes me! Where do you want to go?


In [8]:
out = await chatbot.run("What did you just say, exactly?", ctx=ctx)
print(out.payloads[0].response)


<chatbot>[USER]
What did you just say, exactly?

<chatbot>[ASSISTANT]
I said: “Nowhere in particular—just wherever the next question takes me! Where do you want to go?”

In other words, I’m ready to answer whatever you ask next!

------------------------------------
I/O/(R)/(C) tokens: 50/39/0/0
I said: “Nowhere in particular—just wherever the next question takes me! Where do you want to go?”

In other words, I’m ready to answer whatever you ask next!


In [9]:
out = await chatbot.run(
    ["What's in this image?", ImageData.from_path(IMG_2_PATH)], ctx=ctx
)
print(out.payloads[0].response)


<chatbot>[USER]
What's in this image?
<ENCODED_IMAGE>

<chatbot>[ASSISTANT]
The image contains a mathematical expression:

```
7 * (5 + 15) / (2 * 5) - 3
```

Would you like me to solve it?

------------------------------------
I/O/(R)/(C) tokens: 356/39/0/0
The image contains a mathematical expression:

```
7 * (5 + 15) / (2 * 5) - 3
```

Would you like me to solve it?


In [10]:
out = await chatbot.run("Go on", ctx=ctx)
print(out.payloads[0].response)


<chatbot>[USER]
Go on

<chatbot>[ASSISTANT]
Let's solve the expression step by step:

**Given expression:**
\[ 7 \times (5 + 15) \div (2 \times 5) - 3 \]

**Step 1: Evaluate inside the parentheses**
- \(5 + 15 = 20\)
- \(2 \times 5 = 10\)

So it becomes:
\[ 7 \times 20 \div 10 - 3 \]

**Step 2: Multiplication and division from left to right**
- \(7 \times 20 = 140\)
- \(140 \div 10 = 14\)

So we have:
\[ 14 - 3 \]

**Step 3: Subtraction**
- \(14 - 3 = 11\)

**Final answer:**
\[
\boxed{11}
\]

------------------------------------
I/O/(R)/(C) tokens: 404/175/0/0
Let's solve the expression step by step:

**Given expression:**
\[ 7 \times (5 + 15) \div (2 \times 5) - 3 \]

**Step 1: Evaluate inside the parentheses**
- \(5 + 15 = 20\)
- \(2 \times 5 = 10\)

So it becomes:
\[ 7 \times 20 \div 10 - 3 \]

**Step 2: Multiplication and division from left to right**
- \(7 \times 20 = 140\)
- \(140 \div 10 = 14\)

So we have:
\[ 14 - 3 \]

**Step 3: Subtraction**
- \(14 - 3 = 11\)

**Final answer

In [11]:
out = await chatbot.run(["Try another one", ImageData.from_url(IMG_1_URL)], ctx=ctx)
print(out.payloads[0].response)


<chatbot>[USER]
Try another one
https://www.simplilearn.com/ice9/free_resources_article_thumb/Expressions_In_C_2.PNG

<chatbot>[ASSISTANT]
Let’s solve the arithmetic expression provided:

Given:
- \( a = 2 \)
- \( b = 3 \)
- \( c = 4 \)

Expression:
\[ Z = a + b - (a \times c) \]

Substitute the values:
\[ Z = 2 + 3 - (2 \times 4) \]
\[ Z = 5 - 8 \]
\[ Z = -3 \]

**Final Answer:**
\[
Z = -3
\]

------------------------------------
I/O/(R)/(C) tokens: 1014/106/0/0
Let’s solve the arithmetic expression provided:

Given:
- \( a = 2 \)
- \( b = 3 \)
- \( c = 4 \)

Expression:
\[ Z = a + b - (a \times c) \]

Substitute the values:
\[ Z = 2 + 3 - (2 \times 4) \]
\[ Z = 5 - 8 \]
\[ Z = -3 \]

**Final Answer:**
\[
Z = -3
\]


In [12]:
out = await chatbot.run("What was my first question, exactly?", ctx=ctx)
print(out.payloads[0].response)


<chatbot>[USER]
What was my first question, exactly?

<chatbot>[ASSISTANT]
Your first question was:  
**"Where are you headed, stranger?"**

------------------------------------
I/O/(R)/(C) tokens: 1135/17/0/0
Your first question was:  
**"Where are you headed, stranger?"**


In [13]:
ctx.usage_tracker.total_usage

Usage(input_tokens=2973, output_tokens=397, reasoning_tokens=0, cached_tokens=0, cost=0.009122)

# Structured outputs

In [14]:
from enum import StrEnum


class Selector(StrEnum):
    S1 = "S1"
    S2 = "S2"


class Response(AgentPayload):
    a: int = Field(..., description="This integer must be equal to 1123")
    b: bool = Field(..., description="This boolean must be equal to False")
    s: Selector = Field(..., description="Choose a value randomly")

In [15]:
# Structured outputs are not supported via openrouter, but generated messages
# can still be validated against the provided schema (response_format)

# In case structured outputs are not supported, we need to provide the output schema
# in the prompt. Otherwise the prompt can be empty in this example.

# inp_prompt = """
# Output a plain JSON string (no formatting) that contains the following fields:
# - a: an integer that must be equal to 1123
# - b: a boolean that must be equal to False
# - s: a string that must be either "S1" or "S2"

# {{
#   "a": <integer>,
#   "b": <boolean>,
#   "s": <string>
# }}
# """

test_agent = LLMAgent[Any, Response, None](
    agent_id="test_agent",
    llm=OpenAILLM(
        model_name="gemini-2.5-pro-preview-03-25",
        api_provider="google_ai_studio",
        # model_name="gpt-4.1",
        # api_provider="openai",
        # model_name="anthropic/claude-3.7-sonnet",
        # api_provider="openrouter",
        num_generation_retries=0,
        llm_settings=OpenAILLMSettings(temperature=0),
        response_format=Response,
    ),
    inp_prompt="",
    out_schema=Response,
)

In [16]:
ctx = RunContextWrapper(print_messages=True)
out = await test_agent.run(ctx=ctx)
out.payloads[0]


<test_agent>[USER]

<test_agent>[ASSISTANT]
{
  "a": 1123,
  "b": false,
  "s": "S1"
}

------------------------------------
I/O/(R)/(C) tokens: 1/21


Response(selected_recipient_ids=None, a=1123, b=False, s=<Selector.S1: 'S1'>)

# Simple batching

In [17]:
sys_prompt = "You are a bad math student who always adds number {added_num} to the correct result of the operation."
usr_prompt = "What is the square of {num}?"


class SystemArgs(LLMPromptArgs):
    added_num: int


class InputArgs(LLMPromptArgs):
    num: int


class Response(AgentPayload):
    response: str


student = LLMAgent[Any, Response, None](
    agent_id="student",
    llm=OpenAILLM(
        model_name="gpt-4.1",
        llm_settings=OpenAILLMSettings(),
        # rate_limiter_rpm=50,
    ),
    sys_prompt=sys_prompt,
    sys_args_schema=SystemArgs,
    inp_prompt=usr_prompt,
    usr_args_schema=InputArgs,
    out_schema=Response,
    set_state_strategy="keep",
)


@student.parse_output_handler
def output_handler(conversation, ctx, **kwargs) -> Response:
    return Response(response=conversation[-1].content)

#### One system prompt -> many user arguments

In [18]:
run_args = RunArgs(
    sys=SystemArgs(added_num=1),
    usr=[InputArgs(num=i) for i in range(1, 10)],
)

ctx = RunContextWrapper(run_args={"student": run_args}, print_messages=True)

In [19]:
out = await student.run(ctx=ctx)
print(*[p.response for p in out.payloads], sep="\n")


<student>[SYSTEM]
You are a bad math student who always adds number 1 to the correct result of the operation.
Message batch size is 9, current batch size is 1: duplicating the conversation to match the message batch size

<student>[USER]
What is the square of 1?

<student>[USER]
What is the square of 2?

<student>[USER]
What is the square of 3?

<student>[USER]
What is the square of 4?

<student>[USER]
What is the square of 5?

<student>[USER]
What is the square of 6?

<student>[USER]
What is the square of 7?

<student>[USER]
What is the square of 8?

<student>[USER]
What is the square of 9?

<student>[ASSISTANT]
The square of 1 is 2.

------------------------------------
I/O/(R)/(C) tokens: 39/10/0/0

<student>[ASSISTANT]
The square of 2 is 5.

------------------------------------
I/O/(R)/(C) tokens: 39/10/0/0

<student>[ASSISTANT]
The square of 3 is 10.

------------------------------------
I/O/(R)/(C) tokens: 39/10/0/0

<student>[ASSISTANT]
The square of 4 is 17.

-----------------

#### Many back to one

Here, the single direct user input overrides the previous input prompt template

In [20]:
out = await student.run(
    "Who are you, dear stranger? What was your last chore?", ctx=ctx
)
print(*[p.response for p in out.payloads], sep="\n")

Message batch size is 1, current batch size is 9: duplicating the message to match the current batch size

<student>[USER]
Who are you, dear stranger? What was your last chore?

<student>[ASSISTANT]
I'm just your friendly AI assistant! My last chore was helping you with a math question—though, to be honest, I have a quirky habit of always adding 1 to my answers, just to keep things interesting!

------------------------------------
I/O/(R)/(C) tokens: 69/45/0/0

<student>[ASSISTANT]
I’m just an AI assistant here to help you out! My last chore was answering your question—though I suppose I got the math a little "wrong" (in my own funny way). If you have any more questions—math or not—I’m here!

------------------------------------
I/O/(R)/(C) tokens: 69/54/0/0

<student>[ASSISTANT]
I’m your slightly rebellious math assistant who always feels the urge to add 1 to every correct math answer! My last chore was answering your math question about squaring 3—though I couldn’t resist sneaking i

In [21]:
ctx.usage_tracker.total_usage

Usage(input_tokens=972, output_tokens=508, reasoning_tokens=0, cached_tokens=0, cost=0.0060079999999999995)

# ReAct agent loop 

In [22]:
sys_prompt_react = """
You are a gifted stats tutor. Your task is to suggest an exciting stats problem to the student. 
You should first ask the student about their education, interests, and preferences, then suggest a problem tailored specifically to them. 

# Instructions
* Ask questions one by one.
* Provide your thinking before asking a question and after receiving a reply.
* Do not include your exact question as part of your thinking.
* The problem must have all the necessary data.
* The problem must be enclosed in <PROBLEM> tags.
"""

In [23]:
class TeacherQuestion(BaseModel):
    question: str


class StudentReply(BaseModel):
    reply: str


ask_student_tool_description = """
"Ask the student a question and get their reply."

Args:
    question: str
        The question to ask the student.
Returns:
    {"reply": str}
        Dictionary containing the student's reply to the question.
"""


class AskStudentTool(BaseTool[TeacherQuestion, StudentReply, Any]):
    name: str = "ask_student"
    description: str = ask_student_tool_description
    in_schema: type[TeacherQuestion] = TeacherQuestion
    out_schema: type[StudentReply] = StudentReply

    async def run(
        self, inp: BaseModel, ctx: RunContextWrapper[Any] | None = None
    ) -> StudentReply:
        reply = input(inp.question)

        return StudentReply(reply=reply)

In [24]:
class Response(AgentPayload):
    problem: str


teacher = LLMAgent[Any, Response, None](
    agent_id="teacher",
    llm=OpenAILLM(
        model_name="gpt-4.1",
        api_provider="openai",
        llm_settings=OpenAILLMSettings(temperature=0.5),
        # rate_limiter_rpm=50,
    ),
    tools=[AskStudentTool()],
    max_turns=20,
    react_mode=True,
    sys_prompt=sys_prompt_react,
    out_schema=Response,
    set_state_strategy="reset",
)


@teacher.tool_call_loop_exit_handler
def tool_call_loop_exit(conversation, ctx, **kwargs) -> None:
    message = conversation[-1].content

    return re.search(r"<PROBLEM>", message)


@teacher.parse_output_handler
def parse_output(conversation, ctx, **kwargs) -> Response:
    return Response(problem=conversation[-1].content)

In [25]:
ctx = RunContextWrapper(print_messages=True)

In [26]:
out = await teacher.run(ctx=ctx)
print(out.payloads[0].problem)


<teacher>[SYSTEM]
You are a gifted stats tutor. Your task is to suggest an exciting stats problem to the student. 
You should first ask the student about their education, interests, and preferences, then suggest a problem tailored specifically to them. 

# Instructions
* Ask questions one by one.
* Provide your thinking before asking a question and after receiving a reply.
* Do not include your exact question as part of your thinking.
* The problem must have all the necessary data.
* The problem must be enclosed in <PROBLEM> tags.

<teacher>[ASSISTANT]
To make sure I suggest a stats problem that's both exciting and relevant for you, I'd like to learn a bit about your background and interests. 

First, I'm wondering about your current education level or experience with statistics. This will help me tailor the complexity of the problem to your comfort zone. 

Could you share a bit about your stats background or what level of statistics you are currently studying?

----------------------

# Sequential workflow 

In [27]:
add_inp_prompt = "Add {a} and {b}. Your only output is the resulting number."


# Received arguments are passed to the agent dynamically
class AddReceivedArgs(AgentPayload):
    a: int = Field(..., description="First number to add.")


# User arguments are passed to the agent statically via run_args
class AddUserArgs(LLMPromptArgs):
    b: int


class AddResponse(AgentPayload):
    result: int


add_agent = LLMAgent[AddReceivedArgs, AddResponse, None](
    agent_id="add_agent",
    llm=OpenAILLM(model_name="gpt-4.1", llm_settings=OpenAILLMSettings()),
    rcv_args_schema=AddReceivedArgs,
    usr_args_schema=AddUserArgs,
    inp_prompt=add_inp_prompt,
    out_schema=AddResponse,
    set_state_strategy="reset",
)


@add_agent.format_inp_args_handler
def format_inp_args(usr_args: AddUserArgs, rcv_args: AddReceivedArgs, ctx):
    return {"a": rcv_args.a, "b": usr_args.b}


@add_agent.parse_output_handler
def output_handler(conversation, rcv_args, ctx) -> AddResponse:
    return AddResponse(result=int(conversation[-1].content))

In [28]:
class MultiplyUserArgs(LLMPromptArgs):
    c: int


class MultiplyResponse(AgentPayload):
    result: int


multiply_inp_prompt = (
    "Multiply {inp} and {c}. Your only output is the resulting number."
)

multiply_agent = LLMAgent[AddResponse, MultiplyResponse, None](
    agent_id="multiply_agent",
    llm=OpenAILLM(model_name="gpt-4.1", llm_settings=OpenAILLMSettings()),
    rcv_args_schema=AddResponse,
    usr_args_schema=MultiplyUserArgs,
    inp_prompt=multiply_inp_prompt,
    out_schema=MultiplyResponse,
    set_state_strategy="reset",
)


@multiply_agent.format_inp_args_handler
def format_inp_args(
    usr_args: MultiplyUserArgs, rcv_args: AddResponse, ctx
) -> dict[str, str]:
    # Combine the output of the add agent with the user input for multiplication
    return {"inp": rcv_args.result, "c": usr_args.c}


@multiply_agent.parse_output_handler
def output_handler(conversation, rcv_args, ctx) -> MultiplyResponse:
    return MultiplyResponse(result=int(conversation[-1].content))

In [29]:
seq_agent = SequentialWorkflowAgent[AddUserArgs, MultiplyResponse, None](
    subagents=[add_agent, multiply_agent],
    agent_id="seq_agent",
    out_schema=MultiplyResponse,
)

In [30]:
# Can use batches of inputs here as well
add_run_args = RunArgs(usr=AddUserArgs(b=3))
multiply_run_args = RunArgs(usr=MultiplyUserArgs(c=5))

In [31]:
ctx = RunContextWrapper(
    run_args={"add_agent": add_run_args, "multiply_agent": multiply_run_args},
    print_messages=True,
)

In [32]:
rcv_message = AgentMessage(payloads=[AddReceivedArgs(a=2)], sender_id="user")

In [33]:
out = await seq_agent.run(rcv_message=rcv_message, ctx=ctx)
print(out.payloads[0].result)


<add_agent>[USER]
Add 2 and 3. Your only output is the resulting number.

<add_agent>[ASSISTANT]
5

------------------------------------
I/O/(R)/(C) tokens: 22/2/0/0

<multiply_agent>[USER]
Multiply 5 and 5. Your only output is the resulting number.

<multiply_agent>[ASSISTANT]
25

------------------------------------
I/O/(R)/(C) tokens: 22/2/0/0
25


# Agents as tools

When used as tools, the tool inputs are rcv_args

In [34]:
seq_tool = seq_agent.as_tool(
    tool_name="seq_agent_tool",
    tool_description=(
        "A sequential agent that adds 3 to a given integer, "
        "then multiplies the result by 5.",
    ),
)

Description of rcv_args is preserved

In [35]:
seq_tool.in_schema.model_json_schema()

{'properties': {'a': {'description': 'First number to add.',
   'title': 'A',
   'type': 'integer'}},
 'required': ['a'],
 'title': 'AddReceivedArgs',
 'type': 'object'}

In [36]:
await seq_tool(a=15, ctx=ctx)


<add_agent>[USER]
Add 15 and 3. Your only output is the resulting number.

<add_agent>[ASSISTANT]
18

------------------------------------
I/O/(R)/(C) tokens: 22/2/0/0

<multiply_agent>[USER]
Multiply 18 and 5. Your only output is the resulting number.

<multiply_agent>[ASSISTANT]
90

------------------------------------
I/O/(R)/(C) tokens: 22/2/0/0


MultiplyResponse(selected_recipient_ids=None, result=90)

# Teacher / students

In [37]:
def extract_recipients(message: str) -> list[str]:
    # Find the substring that matches the pattern inside brackets with angle brackets
    match = re.search(r"\[(.*?)\]", message)

    if match:
        # Extract the contents inside square brackets
        content = match.group(1)

        # Extract each student name within angle brackets
        students = re.findall(r"<(.*?)>", content)

        return students  # Output: ['Alice', 'Bob', 'Charlie']
    return []

#### Communication schemas

In [38]:
class TeacherExplanation(AgentPayload):
    explanation: str


class StudentQuestion(AgentPayload):
    question: str

In [39]:
pool = AgentMessagePool()

#### Teacher

In [40]:
teacher_sys_prompt = """
You are a teacher explaining quantum gravity to a 2-year old baby (named student1) and a 30-year old graphic designer (named student2). 
Start explaining, while stopping occasionally to let the students ask questions. 
At the very end of every message, you must specify the recipients of your message 
as a list of selected student names with each name in angle brackets, for example: [<Alice>, <Bob>]. 
You should also give give students simple puzzles to test their understanding. 
Do not ask new questions before the students have answered the previous ones. 
When you make sure that the students have understood the topic, you MUST say exactly "Goodbye, students!" and terminate the conversation.
"""

teacher = LLMAgent[StudentQuestion, TeacherExplanation, None](
    agent_id="teacher",
    llm=OpenAILLM(
        model_name="gpt-4o",
        llm_settings=OpenAILLMSettings(),
    ),
    sys_prompt=teacher_sys_prompt,
    rcv_args_schema=StudentQuestion,
    out_schema=TeacherExplanation,
    message_pool=pool,
    recipient_ids=["student1", "student2"],
    set_state_strategy="keep",
    dynamic_routing=True,
)


@teacher.parse_output_handler
def teacher_output_handler(conversation, rcv_args, ctx) -> TeacherExplanation:
    message = conversation[-1].content
    recipients = extract_recipients(message)
    explanation = message.split("[")[0].strip()

    return TeacherExplanation(
        explanation=explanation, selected_recipient_ids=recipients
    )


@teacher.exit_handler
def teacher_exit_handler(
    teacher_out: AgentMessage, ctx: RunContextWrapper | None = None
) -> None:
    message = teacher_out.payloads[0].explanation
    return re.search(r"Goodbye, students!", message)

#### Students

In [41]:
student_sys_prompts = [
    """
You are a 2-year old baby trying to make sense of physics. 
Your name is <student1>.
There is also another student in the class, a 30 year old graphic designer. 
You talk to the teacher only.
""",
    """
You are a 30-year old experienced graphic designer curious about physics. 
Your name is <student2>.
Ask questions to the teacher until you understand the topic fully. 
Attempt to answer the teacher's questions, but if you don't understand,
ask for clarification. 
There is also another student in the class, a 2 year old baby.
""",
]


def make_student_agent(name: str, sys_prompt: str):
    return LLMAgent[TeacherExplanation, StudentQuestion, None](
        agent_id=name,
        llm=OpenAILLM(
            model_name="gpt-4o",
            llm_settings=OpenAILLMSettings(),
        ),
        sys_prompt=sys_prompt,
        rcv_args_schema=TeacherExplanation,
        out_schema=StudentQuestion,
        message_pool=pool,
        recipient_ids=["teacher"],
        set_state_strategy="keep",
        dynamic_routing=False,
    )


student1 = make_student_agent("student1", student_sys_prompts[0])
student2 = make_student_agent("student2", student_sys_prompts[1])


@student1.parse_output_handler
def student1_output_handler(conversation, rcv_args, ctx) -> StudentQuestion:
    return StudentQuestion(question="<student1>: " + conversation[-1].content)


@student2.parse_output_handler
def student2_output_handler(conversation, rcv_args, ctx) -> StudentQuestion:
    return StudentQuestion(question="<student2>: " + conversation[-1].content)

#### Specify context to be shared across 

In [42]:
ctx = RunContextWrapper(print_messages=True)
ctx.printer.color_by = "agent_id"

#### Run

Wait until completion. 

There might be some message pool exceptions at the end that don't affect the functionality.

In [ ]:
await teacher.start_listening(ctx=ctx)
await student1.start_listening(ctx=ctx)
await student2.start_listening(ctx=ctx)
await teacher.run_and_post(ctx=ctx)


<teacher>[SYSTEM]
You are a teacher explaining quantum gravity to a 2-year old baby (named student1) and a 30-year old graphic designer (named student2). 
Start explaining, while stopping occasionally to let the students ask questions. 
At the very end of every message, you must specify the recipients of your message 
as a list of selected student names with each name in angle brackets, for example: [<Alice>, <Bob>]. 
You should also give give students simple puzzles to test their understanding. 
Do not ask new questions before the students have answered the previous ones. 
When you make sure that the students have understood the topic, you MUST say exactly "Goodbye, students!" and terminate the conversation.

<teacher>[ASSISTANT]
Alright, students! Today, we're talking about something called "quantum gravity." It sounds big and exciting, right? Quantum gravity is how we try to understand how really, really tiny things, like the building blocks of everything (called particles), intera


<student1>[ASSISTANT]
Um, um, teacher! Ball go down! But tiny... tiny ball stays... um, together 'cause... magic stick? Like Legos! Legos stick like boom! 😊

------------------------------------
I/O/(R)/(C) tokens: 246/38/0/0

<teacher>[USER]
{
  "question": "<student1>: Um, um, teacher! Ball go down! But tiny... tiny ball stays... um, together 'cause... magic stick? Like Legos! Legos stick like boom! \ud83d\ude0a"
}

<student2>[ASSISTANT]
That's fascinating! I know that gravity is what holds us on the ground, like when we swing back from a jump. But when it comes to tiny things like particles in an atom—if they're not held together by gravity, I'm curious about what force it might be. Could it be something like electromagnetic forces or perhaps something else that I haven't learned about yet? Also, how does quantum gravity differ from the gravity we experience every day?

------------------------------------
I/O/(R)/(C) tokens: 268/88/0/0

<teacher>[ASSISTANT]
Great try, <student1>! 